In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

### Load data

In [ ]:
train = pd.read_csv('../input/freesound-audio-tagging/train.csv')
sample_submission = pd.read_csv('../input/freesound-audio-tagging/sample_submission.csv')

# Path to files
train_path = '../input/freesound-audio-tagging/audio_train/'
test_path = '../input/freesound-audio-tagging/audio_test/'

train_audio = os.listdir(train_path)
test_audio = os.listdir(test_path)

### Data preprocessing
Generate features with mean and standard deviation as the value input to the model.

In [ ]:
SAMPLE_RATE = 44100

def clean_filename(fname, string):   
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':        
        file_name = string + file_name
    return file_name

# Generate features with mean and standard deviation
def get_mfcc(name, path):
    data, _ = librosa.core.load(path + name, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        mfcc = librosa.feature.mfcc(data, sr = SAMPLE_RATE, n_mfcc=30)
        rms = librosa.feature.rms(data)[0]
        chroma_stft = librosa.feature.chroma_stft(data)[0]
        zcr = librosa.feature.zero_crossing_rate(data)[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(data)[0]
        spectral_centroid = librosa.feature.spectral_centroid(data)[0]
        spectral_contrast = librosa.feature.spectral_contrast(data)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(data)[0]
        spectral_flatness = librosa.feature.spectral_flatness(data)[0]                                  
        mfcc_trunc = np.hstack((np.mean(mfcc, axis=1), np.std(mfcc, axis=1), skew(mfcc, axis = 1), np.max(mfcc, axis = 1), np.median(mfcc, axis = 1), np.min(mfcc, axis = 1)))
        rms_trunc = np.hstack((np.mean(rms), np.std(rms), skew(rms), np.max(rms), np.median(rms), np.min(rms)))
        chroma_stft_trunc = np.hstack((np.mean(chroma_stft), np.std(chroma_stft), skew(chroma_stft), np.max(chroma_stft), np.median(chroma_stft), np.min(chroma_stft)))
        zcr_trunc = np.hstack((np.mean(zcr), np.std(zcr), skew(zcr), np.max(zcr), np.median(zcr), np.min(zcr)))
        spectral_rolloff_trunc = np.hstack((np.mean(spectral_rolloff), np.std(spectral_rolloff), skew(spectral_rolloff), np.max(spectral_rolloff), np.median(spectral_rolloff), np.min(spectral_rolloff)))
        spectral_centroid_trunc = np.hstack((np.mean(spectral_centroid), np.std(spectral_centroid), skew(spectral_centroid), np.max(spectral_centroid), np.median(spectral_centroid), np.min(spectral_centroid)))
        spectral_contrast_trunc = np.hstack((np.mean(spectral_contrast), np.std(spectral_contrast), skew(spectral_contrast), np.max(spectral_contrast), np.median(spectral_contrast), np.min(spectral_contrast)))
        spectral_bandwidth_trunc = np.hstack((np.mean(spectral_bandwidth), np.std(spectral_bandwidth), skew(spectral_bandwidth), np.max(spectral_bandwidth), np.median(spectral_bandwidth), np.min(spectral_bandwidth)))
        spectral_flatness_trunc = np.hstack((np.mean(spectral_flatness), np.std(spectral_flatness), skew(spectral_flatness), np.max(spectral_flatness), np.median(spectral_flatness), np.min(spectral_flatness)))
        
        return pd.Series(np.hstack((mfcc_trunc, rms_trunc, chroma_stft_trunc, zcr_trunc, spectral_rolloff_trunc, spectral_centroid_trunc, spectral_contrast_trunc, spectral_bandwidth_trunc, spectral_flatness_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*210)

In [ ]:
train_data = pd.DataFrame()
train_data['fname'] = train['fname']
train_data = train_data['fname'].progress_apply(get_mfcc, path='../input/freesound-audio-tagging/audio_train/')
print('done loading train mfcc')

In [ ]:
train_data

In [ ]:
train_data['fname'] = train['fname']
train_data['label'] = train['label']
train_data.head()

In [ ]:
train_data.to_csv('../input/testdata/kaggle_train.csv')

### Train Data

In [ ]:
train_data=pd.read_csv('../input/testdata/kaggle_train.csv')

In [ ]:
# Dropping unneccesary columns
X_train = train_data.drop(['fname','label'],axis=1)

In [ ]:
genre_list = train_data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X_train.iloc[:, :-1], dtype = float))

### Test Data

In [ ]:
test_data = pd.DataFrame()
test_data['fname'] = test_audio
test_data = test_data['fname'].progress_apply(get_mfcc, path='../input/freesound-audio-tagging/audio_test/')
print('done loading test mfcc')

In [ ]:
test_data['fname'] = test_audio
test_data['label'] = np.zeros((len(test_audio)))

In [ ]:
test_data

In [ ]:
test_data.to_csv('../input/testdata/kaggle_test.csv', index=False)

In [ ]:
test_data=pd.read_csv('../input/testdata/kaggle_test.csv')

In [ ]:
X_test = test_data.drop(['fname','label'],axis=1)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(np.array(X_train.iloc[:, :-1], dtype = float))
X_test_scaled = scaler.fit_transform(np.array(X_test.iloc[:, :-1], dtype = float))

### Split the data into training and test sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y, test_size=0.2)

In [ ]:
len(y_train)

In [ ]:
len(y_val)

In [ ]:
partial_x_train, x_test, partial_y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

### Create model

In [ ]:
from keras import models
from keras import optimizers
from keras.layers import Dense, Dropout

model = models.Sequential()
model.add(Dense(512, activation='relu', input_shape=(partial_x_train.shape[1],)))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(41, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(partial_x_train,
          partial_y_train,
          epochs=35,
          batch_size=256,
          validation_data=(X_val, y_val))
results = model.evaluate(x_test, y_test)

### Result of training

In [ ]:
results

### Predict Test Data

In [ ]:
predictions=model.predict(X_test_scaled)

In [ ]:
model.save('kaggle_sound_2.h5')

In [ ]:
score = np.max(predictions,axis=1)
score= score.tolist()

In [ ]:
output = np.argmax(predictions, axis=1)
output

In [ ]:
output = np.argmax(predictions, axis=1)
subm = pd.DataFrame()
subm['fname'] = test_audio
subm['label'] = encoder.inverse_transform(output)
subm.to_csv('./submission.csv', index=False)